[Install](https://conda.io/docs/user-guide/install/index.html) Jupyter Notebook.

# $$Python\ Workshop\ 2019$$

Tədbirin gündəliyi:
* Kaggle nədir və Kaggle vasitəsilə neçə data elmini öyrənmək olar;
* Məlumat analizində Python-nun yeri;
* Real nümunə üzərində məlumat analizi;

Real nümunə haqqında:
* **`Data collection / web crawling`** (məlumat toplanması) - ən böyük onlayn iş elanı veb səhifəsindən məlumatların avtomatik götürülməsi və faylda saxlanılması;
* **`Data cleaning`** (məlumat təmizlənməsi) - dəyişənlərin tipinin dəyişdirilməsi, catışmayan məlumatların bərpa olunması, artıq sətrlərin təmizlənməsi;
* **`Feature generation`** (yeni dəyişənlərin yaradılması) - əldə olan məlumatlardan yenisinin yaradılması, misal üçün: elanın mətnindən hansı dildə yazılması;
* **`Data visualizatoin`** (məlumatların təsviri) - məlumatların vizual təqdim olunması;

In [43]:
'#'.join(ch for ch in 'python')

'p#y#t#h#o#n'

In [42]:
list('python')

['p', 'y', 't', 'h', 'o', 'n']

In [41]:
for ch in 'python':
    print(ch)

p
y
t
h
o
n


## $$The\ Zen\ of\ Python$$

In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


Understanding `The Zen of Python` via [examples](https://artifex.org/~hblanks/talks/2011/pep20_by_example.html).

## $$1.\ Import\ packages$$

In [2]:
%matplotlib inline

import requests
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import langdetect

from wordcloud import WordCloud

## $$2.\ Import\ data\ from\ website$$

In [ ]:
# if `jobs.csv` exist don't extract data from website

file_name = 'jobs.csv'
url = 'http://jobsearch.az/'

if not os.path.isfile(file_name):
    print("File doesn't exit. Extract from website.")
    html = requests.get(url).content
    df_list = pd.read_html(html)
    jobs = df_list[9]
    # export to csv
    jobs.to_csv('jobs.csv', index=False)
else:
    print("File exist. Read from file.")
    jobs = pd.read_csv(file_name)

Remove first column and put first row as column.

In [ ]:
# jobs.drop(columns=[jobs.columns[0]], axis=1, inplace=True) # remove first column
jobs.columns = jobs.iloc[0, :] # rename column to first row
jobs = jobs.iloc[1:, :] # remove first row

## $$3.\ Data\ cleaning$$

In [ ]:
print('Dimension of the dataset :', jobs.shape)
jobs.head()

Number of missing values per column.

In [ ]:
jobs.isnull().sum()

Get types per column.

In [ ]:
jobs.get_dtype_counts()

## $$4.\ Feature\ generation$$

Dummy variable to seperate new and old features.

In [ ]:
def is_new_job(pos):
    '''Return dummy variable about '''
    return 'New' if pos.split()[-1] == '(new)' else 'Old'

In [ ]:
jobs = jobs.assign(is_new = jobs.POSITION.apply(lambda x: is_new_job(x)))

Remove `(new)` tag from job position.

In [ ]:
def clean_position(pos):
    '''Cleaning Positions'''
    pos = pos.lower()
    pos_clean = ' '.join(w for w in pos.split() if w != '(new)')
    pos_clean = pos_clean.strip()
    
    return pos_clean

In [ ]:
jobs = jobs.assign(POSITION = jobs.POSITION.apply(lambda x: clean_position(x)))

Convert string to date type. Add datetime variables:

* Weekday
* Month

In [ ]:
jobs.POSTED = pd.to_datetime(jobs.POSTED) # convert to date type
jobs.DEADLINE = pd.to_datetime(jobs.DEADLINE) # convert to date type

month_name = {
    1 : 'Yanvar',
    2 : 'Fevral',
    3 : 'Mart',
    4 : 'Aprel',
    5 : 'May',
    6 : 'Iyun',
    7 : 'Iyul',
    8 : 'Avqust',
    9 : 'Sentyabr',
    10 : 'Oktyabr',
    11 : 'Noyabr',
    12 : 'Dekabr'
}

jobs = jobs.assign(posted_month = jobs.POSTED.dt.month.map(month_name))
jobs = jobs.assign(posted_weekday = jobs.POSTED.dt.weekday_name)

Find out `position` language, in which language the position was published.

In [ ]:
def convert_language(lang):
    '''Convert language name and group them'''
    if lang == 'tr':
        return 'az'
    elif lang in ['en', 'ru']:
        return lang
    else:
        return 'other'

In [ ]:
jobs = jobs.assign(position_lang = jobs.POSITION.apply(lambda x: langdetect.detect(x)))
jobs = jobs.assign(position_lang = jobs['position_lang'].map(convert_language))

In [ ]:
jobs.head()

In [ ]:
jobs.position_lang.value_counts(normalize=True) * 100

In [ ]:
jobs[jobs.position_lang == 'other'].head()

Get type of `employer`

In [ ]:
# get last word from EMPLOYER
emp_type = jobs.EMPLOYER.apply(lambda x: x.split()[-1])

In [ ]:
emp_type.value_counts(normalize=False).head(10) # top 10 employer type

We have the most popular employer type.

In [ ]:
def get_employer_type(employer):
    '''Get employer type'''
    emp_types = ['LLC', 'MMC', 'ASC', 'Group', 'Holding', 'Bank', 'LTD']
    last_word = employer.split()[-1]
    return last_word if last_word in emp_types else 'Other'

In [ ]:
jobs = jobs.assign(employer_type = jobs.EMPLOYER.apply(lambda x: get_employer_type(x)))

In [ ]:
jobs.employer_type.value_counts(normalize=True) * 100

## $$5.\ Visualization$$

In [ ]:
jobs.head()

Timeline of job publishment.

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
jobs.groupby(['POSTED'])['POSTED'].count().plot(ax=ax)

In [ ]:
jobs.POSTED.value_counts().head() # top 6 date with number of job ads

The most frequent `Employer` with jobs ads.

In [ ]:
jobs.EMPLOYER.value_counts().head(10)

The most frequent `Position`.

In [ ]:
jobs.POSITION.value_counts().head(10)

When the most popular day to publish job description.

In [ ]:
sns.countplot(y=jobs.posted_weekday, 
              order = jobs.posted_weekday.value_counts().index)

Frequency of the job insertion.

In [ ]:
sns.countplot(jobs.is_new)

Frequency of the `employer type`

In [ ]:
sns.countplot(y=jobs.employer_type,
              order=jobs.employer_type.value_counts().index)

Frequency of the `position language`

In [ ]:
sns.countplot(y=jobs.position_lang,
              order=jobs.position_lang.value_counts().index)

In [ ]:
jobs.head()

## Word Cloud of Position

In [ ]:
def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        max_words=75,
        max_font_size=40, 
        scale=7,
        random_state=1 # chosen at random by flipping a coin; it was heads
    ).generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

### POSITION

`AZ`

### `long version`

In [ ]:
filter_by_lang = jobs.POSITION[jobs.position_lang == 'az']
text = []

# seperate each word from `position`
for sen in filter_by_lang.str.split(): # for each sentence
    for w in sen: # for each word
        text.append(w)

# combine into a string
text = ' '.join(text)

# word cloud
show_wordcloud(text)

### `short verion`

In [ ]:
text = ' '.join(w for sen in jobs.POSITION[jobs.position_lang == 'az'].str.split() for w in sen)
show_wordcloud(text)

`EN`

In [ ]:
text = ' '.join(w for sen in jobs.POSITION[jobs.position_lang == 'en'].str.split() for w in sen)
show_wordcloud(text)

`RU`

In [ ]:
text = ' '.join(w for sen in jobs.POSITION[jobs.position_lang == 'ru'].str.split() for w in sen)
show_wordcloud(text)

`OTHER`

In [ ]:
text = ' '.join(w for sen in jobs.POSITION[jobs.position_lang == 'other'].str.split() for w in sen)
show_wordcloud(text)

## EMPLOYER

In [ ]:
text = ' '.join(w for sen in jobs.EMPLOYER.str.split() for w in sen)
show_wordcloud(text)